# ITSM Icident Faulty System Prediction

Dataset: [Incident management process enriched event log Data Set](https://archive.ics.uci.edu/ml/datasets/Incident+management+process+enriched+event+log#)

## Initialise work space

In [3]:
# common python modules
import math
import re
import string

# Math and Analytics modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# NLP
from flashtext import KeywordProcessor
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from textblob import TextBlob, Word
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans

## Load Data

In [4]:
df_imraw = pd.read_csv("data/incident_event_log.csv")

In [5]:
df_imraw.head()

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at
0,INC0000045,New,True,0,0,0,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
1,INC0000045,Resolved,True,0,0,2,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
2,INC0000045,Resolved,True,0,0,3,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
3,INC0000045,Closed,False,0,0,4,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
4,INC0000047,New,True,0,0,0,True,Caller 2403,Opened by 397,29/2/2016 04:40,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 81,1/3/2016 09:52,6/3/2016 10:00


In [9]:
df_imraw.head(3).transpose()

,0,1,2
number,INC0000045,INC0000045,INC0000045
incident_state,New,Resolved,Resolved
active,True,True,True
reassignment_count,0,0,0
reopen_count,0,0,0
sys_mod_count,0,2,3
made_sla,True,True,True
caller_id,Caller 2403,Caller 2403,Caller 2403
opened_by,Opened by 8,Opened by 8,Opened by 8
opened_at,29/2/2016 01:16,29/2/2016 01:16,29/2/2016 01:16
